In [ ]:
"""
1. Get X and y
2. Split into training and testing data
3. Reshape X -> (samples, channels, x-axis, y-axis)
   Reshape y -> (sampples, x-axis * y-axis)
4. Create ReconNet Units
   Figure out the zero padding post each step to maintain a 33 * 33 filter size 
"""
import os
import joblib
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

np.random.seed(7)
%matplotlib inline

def get_data_tfformat(url_data_X, url_data_y):
    print ('0. --------------------- USING TF FORMAT ---------------------  ')
    with open(url_data_X, 'rb') as handle:
        X = joblib.load(handle)
        
    with open(url_data_y, 'rb') as handle:
        y = joblib.load(handle)
    
    
    idxs = np.random.randint(0, len(X), 3)
    
    f, axarr = plt.subplots(1,3)
    axarr[0].imshow(X[idxs[0]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[1].imshow(X[idxs[1]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[2].imshow(X[idxs[2]].reshape((33,33)), cmap = plt.cm.gray)
    
    f, axarr = plt.subplots(1,3)
    axarr[0].imshow(y[idxs[0]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[1].imshow(y[idxs[1]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[2].imshow(y[idxs[2]].reshape((33,33)), cmap = plt.cm.gray)
    
    X = X.reshape((X.shape[0], 1, 33, 33))
    print ('1. X:', X.shape, ' y:', y.shape)
    return X , y

def get_data_theanoformat(url_data_X, url_data_y):
    print ('0. --------------------- USING THEANO FORMAT ---------------------  ')
    with open(url_data_X, 'rb') as handle:
        X = joblib.load(handle)
        X = X.reshape((X.shape[0], 33, 33, 1))
        
    with open(url_data_y, 'rb') as handle:
        y = joblib.load(handle)
    
    
    idxs = np.random.randint(0, len(X), 3)
    
    f, axarr = plt.subplots(1,3)
    axarr[0].imshow(X[idxs[0]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[1].imshow(X[idxs[1]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[2].imshow(X[idxs[2]].reshape((33,33)), cmap = plt.cm.gray)
    
    f, axarr = plt.subplots(1,3)
    axarr[0].imshow(y[idxs[0]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[1].imshow(y[idxs[1]].reshape((33,33)), cmap = plt.cm.gray)
    axarr[2].imshow(y[idxs[2]].reshape((33,33)), cmap = plt.cm.gray)
    
    print ('1. X:', X.shape, ' y:', y.shape)
    return X , y

if __name__ == "__main__" :
    url_data_X = '../data/data_patches_X.gz'
    url_data_y = '../data/data_patches_y.gz'
    # X, y = get_data_theanoformat(url_data_X, url_data_y)
    X, y = get_data_tfformat(url_data_X, url_data_y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0)
    print ('1. X_train:', X_train.shape, ' y_train:', y_train.shape)
    print ('1. X_test:', X_test.shape, ' y_test:', y_test.shape)

In [ ]:
# %load_ext autoreload
# %autoreload 2

def check_gpu():
    import os
    print ('0. Envs : CUDA_HOME', os.environ['CUDA_HOME'])
    print ('0. Envs : CUDA_ROOT',os.environ['CUDA_ROOT'])
    print ('0. Envs : LD_LIBRARY_PATH:', os.environ['LD_LIBRARY_PATH'])
    print ('0. Envs : PATH (containing cuda)', [each for each in os.environ['PATH'].split(':') if each.find('cuda') > -1])

    from keras import backend as K
    K.clear_session()
    
    print ('0. Keras backend:', K.backend())
    if K.backend() == 'tensorflow':
        from tensorflow.python.client import device_lib
        devices = device_lib.list_local_devices()
        for device in devices:
            print ('0. TensorFlow Devices:', str(device).replace('\n',''))
        print ('\n')
        
        return 1 if len(devices) > 1 else 0
    
    else:
        return 0

def model_train(X_train, y_train, model_filename_weights, epochs = 1, batch_size = 32, callbacks_list=[]):
    ## CNN STRUCTURE ###
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten
    from keras.layers import Convolution2D, MaxPooling2D
    
    model = Sequential()
    
    model.add(Convolution2D(64, kernel_size=(11, 11), activation='relu', use_bias = True
                                , input_shape=(1,33,33), data_format='channels_first', padding='same')
                                # , input_shape=(33,33,1), data_format='channels_last', padding='same')
                            )
    print ('First layer:', model.output_shape)
    # print ('First layer:', model.layers[0].output_shape)

    model.add(Convolution2D(32, kernel_size=(1, 1), activation='relu', use_bias = True, padding='same'))

    model.add(Convolution2D(1, kernel_size=(7, 7), activation='relu', use_bias = True, padding='same'))

    model.add(Convolution2D(64, kernel_size=(11, 11), activation='relu', use_bias = True, padding='same'))

    model.add(Convolution2D(32, kernel_size=(1, 1), activation='relu', use_bias = True, padding='same'))

    model.add(Convolution2D(1, kernel_size=(7, 7), activation='relu', use_bias = True, padding='same'))
    
    model.add(Flatten())
    
    ## CHECK IF NEW / OLDER MODEL ##
    if model_filename_weights == '':
        pass
    else:
        model.load_weights(model_filename_weights)

    ## CONTINUE WITH COMPILATION AND TRAINING ##
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

    model.summary()

    X_train_model = X_train.astype('float32')
    X_train_model /= 255  
    y_train_model = y_train.astype('float32')
    y_train_model /= 255

    history = model.fit(X_train_model, y_train_model, batch_size=batch_size, epochs = epochs
                      , shuffle=True
                      , validation_split=0.1
                      , callbacks=callbacks_list
                      , verbose=1)
        
    return model, history
    

def model_validate(model, X_test, y_test):
    print ('\n4. --------------------------------------> Validating model')
    X_test_model = X_test.astype('float32')
    X_test_model /= 255  
    y_test_model = y_test.astype('float32')
    y_test_model /= 255
    metrics = model.evaluate(X_test_model, y_test_model, verbose=1)
    print ('\nValidation Metrics:', metrics)    

def model_disk(action, filename_model_arch, filename_model_weights, model=''):
    from keras.models import model_from_json
    
    print ('\n3. --------------------------------------> Model on Disk')
    if action == 'save':
        with open(filename_model_arch, "w") as handle:
            handle.write(model.to_json())
        model.save_weights(filename_model_weights)
        print("\nSaved model to disk")
        
    elif action == 'load':
        json_file = open(filename_model_arch, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.load_weights(filename_model_weights)
        print("Loaded model from disk")
    
    return model

def model_run(MODEL_NEW, TEST, EPOCHS, BATCH_SIZE, model_filename_weights_best):
    ## CHECK TENSORFLOW DEVICES ##
    if (check_gpu()):
        from keras.callbacks import ModelCheckpoint
        
        ## 1.1 TRAIN MODEL : NEW MODEL OR OLD ##
        if MODEL_NEW == 1:
            print ('2. --------------------------------------> Fresh model')
            model_filename_weights_best = ''
        else:
            print ('2. --------------------------------------> Updating model(',model_filename_weights_best,')')
            model_filename_weights_best = model_filename_weights_best

        ## 1.2 TRAIN MODEL : CREATE CHECKPOINTING INFO ##
        model_filename_weights = "model/index_network_euclidean_keras_weights_{epoch:02d}_{val_acc:.2f}.hdf5"
        # model_filename_weights = "model/index_network_euclidean_keras_weights_{epoch:02d}.h5"
        checkpoint = ModelCheckpoint(model_filename_weights, monitor='val_acc', mode='auto'
                                     # , save_best_only = True
                                     , save_best_only = False
                                     , save_weights_only = True, verbose=0.1)
        callbacks_list = [checkpoint]

        ## 1.3 TRAIN MODEL : TEST OR PROD MODEL ##
        if TEST == 1:
            model, history = model_train(X_train[:BATCH_SIZE*3], y_train[:BATCH_SIZE*3], model_filename_weights_best, epochs = EPOCHS, batch_size = BATCH_SIZE
                                , callbacks_list = callbacks_list)
        else:
            model, history = model_train(X_train, y_train, model_filename_weights_best, epochs = EPOCHS, batch_size = BATCH_SIZE
                                , callbacks_list = callbacks_list)

        ## 1.4 TRAIN MODEL : SAVE MODEL AND WEIGHTS FOR ANY FUTURE REFERENCES  ##
        filename_model_arch = 'model/index_network_euclidean_keras_model.json'
        filename_model_weights = 'model/index_network_euclidean_keras_weights.h5'
        model = model_disk('save', filename_model_arch, filename_model_weights, model)
        model_validate(model, X_test, y_test)
        
        return model
    
    else:
        print ('0. GPU not available :( ')
        return None

if __name__ == "__main__":
    pass
    # check_gpu()
    
#     MODEL_NEW = 0
#     TEST = 0
#     EPOCHS = 60
#     BATCH_SIZE = 128
    
#     if MODEL_NEW == 1:
#         model_filename_weights_best = ''
#     else:
#         model_filename_weights_best = sorted(glob('model/index_network_euclidean_keras_weights_*')
#                                              , key=os.path.getmtime)[-1]
#         # model_filename_weights_best = 'model/index_network_euclidean_keras_weights_09.h5'
    
#     model = model_run(MODEL_NEW, TEST, EPOCHS, BATCH_SIZE, model_filename_weights_best)
        

In [ ]:
import joblib
import numpy as np
import matplotlib.pyplot as plt

def get_data_random(url_data_X, url_data_y):
    with open(url_data_X, 'rb') as handle:
        X = joblib.load(handle)
        
    with open(url_data_y, 'rb') as handle:
        y = joblib.load(handle)
    
    rand_idx = np.random.randint(0, len(X))
    
    return X[rand_idx], y[rand_idx]
    

if __name__ == "__main__":
    model = model_disk('load', 'model/index_network_euclidean_keras_model.json'
                               , 'model/index_network_euclidean_keras_weights.h5'
                               , ''
                       )
    url_data_X = '../data/data_patches_X.gz'
    url_data_y = '../data/data_patches_y.gz'
    X_rand, y_rand = get_data_random(url_data_X, url_data_y)
    print (X_rand.shape, y_rand.shape)
    
    X_predict = model.predict(X_rand.reshape(1,1,33,33))
    print ('X_predict:', X_predict.shape)
    
    f, axarr = plt.subplots(1,3)
    axarr[0].imshow(X_rand.reshape(33,33), cmap = plt.cm.gray)
    axarr[1].imshow(y_rand.reshape(33,33), cmap = plt.cm.gray)
    axarr[2].imshow(X_predict.reshape(33, 33), cmap = plt.cm.gray)

In [ ]:
from keras import backend as K
K.function([K.learning_phase(), model.layers[0].input], model.layers[2].output)